In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import Counter
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torchvision.transforms.functional as TF
import torchvision.models as models

In [2]:
import os
import cv2
import numpy as np
# Define transformations for the training and testing sets
transform = transforms.Compose([
    transforms.ToTensor()
])

def load_images_from_folder(data_dir, num_channels, width, height, labels):
  """Loads images from a folder with 5 multichannel inputs.

  Args:
    data_dir: Path to the data directory.
    num_channels: Number of channels for each image (1 for grayscale, 3 for RGB).
    width: Target width for resizing images.
    height: Target height for resizing images.
    labels: List of class labels.

  Returns:
    X: A numpy array of shape (num_images, num_channels * 5, height, width).
    y: A numpy array of shape (num_images,) containing class labels.
  """

  X = []
  y = []
  extension = [".png", ".jpg", ".jpeg"]
  for label_index, label_dir in enumerate(sorted(os.listdir(data_dir))):
    if not label_dir.startswith("."):
      label_path = os.path.join(data_dir, label_dir)
      i = 0
      channel_data = []  # List to store data for all 5 channels

      for fname in sorted(os.listdir(label_path)):
        if any(fname.endswith(ext) for ext in extension) and not fname.startswith("."):
          i += 1
          path = os.path.join(label_path, fname)
          img = cv2.imread(path, cv2.IMREAD_COLOR if num_channels == 3 else cv2.IMREAD_GRAYSCALE)
          img = cv2.resize(img, (width, height))
          channel_data.append(img)

          # Check if all 5 channels are loaded, then process and append
          if i % 5 == 0:
            if num_channels == 1:
              feature = np.stack(channel_data, axis=-1)
            elif num_channels == 3:
              feature = np.concatenate(channel_data, axis=-1)
            # Reshape to (num_channels * 5, height, width) before appending to X
            feature = np.transpose(feature, (2, 0, 1))
            X.append(feature)
            y.append(label_index)
            channel_data = []  # Reset channel data list

  return np.asarray(X), np.asarray(y)

In [3]:
A2_1train_dir ="/kaggle/input/vibeid-a1/A1/train"
A2_1test_dir = "/kaggle/input/vibeid-a1/A1/test"

In [ ]:
num_channels = 3
width = 128
height =128
num_classes = 100
labels = sorted(os.listdir(A2_1train_dir))[:num_classes]  # Assuming labels are sorted alphabetically
print(labels)
X_train, y_train = load_images_from_folder(A2_1train_dir, num_channels, width, height, labels)
X_test, y_test = load_images_from_folder(A2_1test_dir, num_channels, width, height, labels)

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming X_train, y_train, X_test, and y_test are numpy arrays or PyTorch tensors

# Convert train and test data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Assuming y_train is categorical
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)  # Assuming y_test is categorical
num_epochs = 10
batch_size = 16
# Create DataLoaders with rearranged data
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

In [ ]:
print("Train data shape (X_train):", X_train_tensor.shape)
print("Train data labels shape (y_train):", y_train_tensor.shape)

print("Test data shape (X_test):", X_test_tensor.shape)
print("Test data labels shape (y_test):", y_test_tensor.shape)
import torch
from collections import Counter

# Assuming y_train_tensor and y_test_tensor are already defined as tensors of labels

# Count the number of samples per class in the training set
train_class_counts = Counter(y_train_tensor.numpy())
print("Training set class distribution:")
for class_label, count in train_class_counts.items():
    print(f"Class {class_label}: {count} samples")

# Count the number of samples per class in the test set
test_class_counts = Counter(y_test_tensor.numpy())
print("\nTest set class distribution:")
for class_label, count in test_class_counts.items():
    print(f"Class {class_label}: {count} samples")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        resnet =  models.resnet50(pretrained=False)
        for param in resnet.parameters():
          param.requires_grad = True
        resnet.conv1 =  nn.Conv2d(15, 64, kernel_size=7, stride=2, padding=3)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Sequential(nn.Linear(num_ftrs,100))
        self.resnet = resnet
    def forward(self, x):
        return self.resnet(x)
custom_resnet = CustomResNet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_resnet.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.01, patience=3, verbose=True)
optimizer =  optim.Adam(custom_resnet.parameters())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_resnet.to(device)
from torch.optim import lr_scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
num_epochs = 100
def train_and_test_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total

#         print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}")

        # Testing phase
        model.eval()  # Set the model to evaluation mode
        test_correct = 0
        test_total = 0
        test_loss = 0.0

        with torch.no_grad():  # Enable gradient computation during inference
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        test_loss /= len(test_loader)
        test_accuracy = test_correct / test_total

        print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        # Update learning rate scheduler
        if scheduler is not None:
            scheduler.step(test_loss)

    print("Training completed!")
# Train and test the model
train_and_test_model(custom_resnet, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs)
print("Training completed!")